In [1]:
import csv
import pandas as pd
import urllib
import urllib2
import datetime
import numpy as np;

In [2]:
df = pd.read_csv('../Data/companylist.csv')
stock_tickers = df.Symbol

In [3]:
##  Calculate moving average for all points over n data points in a list,
##  then also removes n - 1 data points and returns that as a separate list

def calculateMovingAverageList(data, n):    
    currentAverage = 0;
    internalValues = [];
    movingAverage = [];

    for i in range(n):
        # Append all values to internalValues;
        # print i;
        internalValues.append(data[i]);
        
    currentAverage = np.mean(internalValues);
    movingAverage.append(currentAverage);
        
    for i in range(n + 1, len(data)):
        # print i;
        del internalValues[0]; # Delete first index
        internalValues.append(data[i]); # Add data at i;
        currentAverage = np.mean(internalValues);
        movingAverage.append(currentAverage);
        
    return movingAverage, data[n:];

In [4]:
##  Append 0 data

def fixTimeData(timeList, n, data, movingAverage):
    for i in range(n):
        timeList.append(i);
        movingAverage.insert(0, 0);
        
    for j in range(i, len(data) - 1):
        timeList.append(j)
        
    return movingAverage;

In [5]:
def calculateSlope(movingAverageList):
    dX = 1.0;
    dY = [];
    
    for i in range(len(movingAverageList) - 1):
        dY.append(movingAverageList[i + 1] - movingAverageList[i]);
        
    dY.append(movingAverageList[i + 1]);
    slope = [dy / dX for dy in dY];
    return slope;

In [6]:
final_results = {};

In [9]:
import pickle
results_from_script = pickle.load( open( "../Python/final_results_market.p", "rb" ) )

In [21]:
for stock in range(0, len(stock_tickers)):
    print "Currently calculating for stock: " + stock_tickers[stock] + " (progress: " + str(stock) + " of " + str(len(stock_tickers)) + ")"
    dataDict = {};
    times = [];
    closing = [];
    continueBoolean = False;

    with open('../Data_From_Script/getprices_' + stock_tickers[stock] + '_long.txt', 'rb') as csvfile:
        pricereader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for i, row in enumerate(pricereader):
            if i > 6:
                try:
                    dataDict[i] = row;
                    values = row[0].split(',');
                    times.append(values[0]);
                    closing.append(float(values[1]));
                except IndexError:
                    continueBoolean = True;
                    continue;
                    
    if continueBoolean == True:
        continue;
                
    timesUTC = [];

    currUnixTime = 0;
    currTime = None;

    for i in range(len(times)):
        if times[i][0] == 'a':
            #print i;
            currUnixTime = int(times[i][1:]);
            #print currUnixTime;
            currTime = datetime.datetime.utcfromtimestamp(currUnixTime)
            #print currTime
            timesUTC.append(currTime);
        else:
            #print currUnixTime
            timesUTC.append(datetime.datetime.utcfromtimestamp(currUnixTime + int(times[i])));
            
    if len(closing) == 0:
        continue;

    try:
        movingAverage78, shorterData78 = calculateMovingAverageList(closing, 78);
        timeStamp78 = timesUTC[78:];
    except IndexError:
        continue;
    
    movingAverage39, shorterData39 = calculateMovingAverageList(closing, 39);
    timeStamp39 = timesUTC[39:];
    
    movingAverage20, shorterData20 = calculateMovingAverageList(closing, 20);
    timeStamp20 = timesUTC[20:];
    
    movingAverage5, shorterData5 = calculateMovingAverageList(closing, 5);
    timeStamp5 = timesUTC[5:];
    
    fixTime5 = [];
    fixTime20 = [];
    fixTime39 = [];
    fixTime78 = [];

    movingAverage5 = fixTimeData(fixTime5, 5, closing, movingAverage5);
    movingAverage20 = fixTimeData(fixTime20, 20, closing, movingAverage20);
    movingAverage39 = fixTimeData(fixTime39, 39, closing, movingAverage39);
    movingAverage78 = fixTimeData(fixTime78, 78, closing, movingAverage78);
    
    slope78 = calculateSlope(movingAverage78);
    slope39 = calculateSlope(movingAverage39);
    slope20 = calculateSlope(movingAverage20);
    slope5 = calculateSlope(movingAverage5);
    
    buyPoints = [];
    errorBoolean = False;
    
    for i in range(len(slope78)):
        try:
            if (slope78[i] >= 0) and (slope39[i] > 0) and (slope20[i] > 0) and (slope5[i] > 0) and (closing[i] > movingAverage78[i]):
                buyPoints.append(i);
        except IndexError:
            errorBoolean = True;
            continue;
            
    if errorBoolean == True:
        continue;
            
    sellPoints = [];

    for i in range(len(slope78)):
        if (slope78[i] < 0) and (slope39[i] < 0) and (slope20[i] < 0) and (slope5[i] < 0) and (closing[i] < movingAverage78[i]):
            sellPoints.append(i);
            
    ##  Find one buyPoint and one sellPoint

    buy_sell_Points_tuple_list = []

    for buyPoint in buyPoints:
        buy_sell_Points_tuple_list.append((buyPoint, "buy"));

    for sellPoint in sellPoints:
        buy_sell_Points_tuple_list.append((sellPoint, "sell"));
        
    buy_sell_Points_tuple_list.sort(key=lambda tup: tup[0])  # sorts in place
    
    buy_list = [];
    sell_list = [];

    isBuy = True;

    for value in buy_sell_Points_tuple_list:
        if isBuy == True and value[1] == 'buy':
            buy_list.append(value[0]);
            isBuy = False;
        elif isBuy == False and value[1] == 'sell':
            sell_list.append(value[0]);
            isBuy = True;
            
    ##  Calculate earnings (from just sell minus buy)
    earnings = [];
    number_of_transactions = 0;

    for i in range(len(sell_list)):
        earnings.append(closing[sell_list[i]] - closing[buy_list[i]]);
        number_of_transactions += 1;

    for i in range(1, len(buy_list)):
        # print buy_list[i], sell_list[i - 1];
        # print closing[buy_list[i]], closing
        earnings.append(-(closing[buy_list[i]] - closing[sell_list[i - 1]]));
        number_of_transactions += 1;
        
    price_of_transactions = - 7 * number_of_transactions * 2;

    ## Net sum with number_of_shares amount of stock
    
    if len(buy_list) == 0:
        number_of_shares = 0;
    else:
        number_of_shares = 10000 / closing[buy_list[0]];

    ## Percentage gain
    if len(buy_list) == 0:
        starting_amount = 0;
    else:
        starting_amount = closing[buy_list[0]] * number_of_shares + 7;
    
    final_amount = starting_amount + np.sum(earnings) * number_of_shares - price_of_transactions;
    
    final_results[stock_tickers[stock]] = (buy_list, sell_list, number_of_shares, earnings, starting_amount, final_amount);

Currently calculating for stock: PIH (progress: 0 of 341)
Currently calculating for stock: TURN (progress: 1 of 341)
Currently calculating for stock: FLWS (progress: 2 of 341)
Currently calculating for stock: FCCY (progress: 3 of 341)
Currently calculating for stock: SRCE (progress: 4 of 341)
Currently calculating for stock: VNET (progress: 5 of 341)
Currently calculating for stock: TWOU (progress: 6 of 341)
Currently calculating for stock: JOBS (progress: 7 of 341)
Currently calculating for stock: CAFD (progress: 8 of 341)
Currently calculating for stock: EGHT (progress: 9 of 341)
Currently calculating for stock: AVHI (progress: 10 of 341)
Currently calculating for stock: SHLM (progress: 11 of 341)
Currently calculating for stock: AAON (progress: 12 of 341)
Currently calculating for stock: ABAX (progress: 13 of 341)
Currently calculating for stock: ABEO (progress: 14 of 341)
Currently calculating for stock: ABEOW (progress: 15 of 341)
Currently calculating for stock: ABIL (progress: 1

In [22]:
net_earnings = [];

for key in final_results:
    net_earnings.append(final_results[key][5] / final_results[key][4]);

In [26]:
len(final_results)

2155

In [27]:
## Mean of net earnings
print np.nanmean(net_earnings)

1.12085409012


In [24]:
import matplotlib;
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: MacOSX


In [ ]:
net_earnings.index(3.446647479654608)

In [ ]:
final_results.keys()[140]

In [29]:
len(net_earnings)

2155

In [28]:
plt.hist(net_earnings, normed=True, bins=30)
plt.ylabel('Percentage Returns');

ValueError: max must be larger than min in range parameter.

#  Data from:
http://finance.google.com/finance/getprices?i=600&p=30d&f=d,o,h,l,c,v&df=cpct&q=AMD